# Home Credit Default Risk

# Overview

### The Objective
The Home Credit Default Risk project is aimed to predict which clients of loan providers are capable of repaying their loans. Home Credit's goal is the ensure each client has appropriate limits set for principal, maturity, and repayment calendars to empower consumers by identifying repayment abilities.

During this project, we are going to explore several files of data to better understand exactly what pre-existing information we have collected for each consumer, analyze trends, and explore various modeling techniques to predict whether or not the client is likely to default on their payments.

We found early on that the Gradient Boosted Tree models have consistently outperformed other classification models such as Logistic Regression, Classification Trees, and Random Forest Classifiers. After the inital baseline model, the following submissions are primarily using the Gradient Boosted Tree with variations of new features from supplementary datasets along with created features we have engineered.

### The Evaluation
We are measuring the success of our predictive models abilities based on the Receiver Operator Characteristic Area Under the Curve, also known as the ROC-AUC score. The primary measurements accounted for in the ROC-AUC metric are the True-Positive predictions and False-Positive Predictions. Scoring is on a 0 to 1 scale, where 1 represents the models ability to perfectly distinguish our target variable from the input features, in this case, those who are likely to dafault on their loan repayments.

### Conclusion
While this task proved to be rather challenging, it was a great experience working with large volume datasets with largely imbalanced target variables. There are many different approaches that can be explored in these situations such as oversampling or undersampling, but one of the biggest factors in our progress has been thoughtful feature engineering and advanced modeling techniques such as hyperparameter tuning.

As a result of our final model, we were able to achieve an AUC score of **78.425%** which was ~**1.6%** lower than the top public score at the time of the competition.

I think it is valuable to reflect on opportunities that we had when building our models. First, I think the model training would have likely yielded better results if we eliminated outliers in our dataset earlier in the EDA process before determining the features. Second, with several datasets we noticed a large volume of categorical variables; our use of OneHotEncoding would cause several of these features to not even be considered due to dimensionality concerns of our dataset. Our final model included a total of 199 features (including OneHot Encoding), and the model from the week prior resulted in 1% less AUC score with half the amount of features. With the goal of parsimony in mind, reducing the number of features would likely be a greater benefit in the long run.

# Model Submission Steps

## Import Libraries

Change as needed. Once completed, remove un-needed libraries and remove this comment

In [1]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier, plot_importance

import joblib

import warnings

warnings.filterwarnings("ignore")

## Load Preprocessor and Model

change cells below to most recent model etc. and remove this comment

In [2]:
preprocessor = joblib.load('/kaggle/input/hcdr-model-v6/HCDR_preprocessor_06.joblib')

model = XGBClassifier()
model.load_model('/kaggle/input/hcdr-model-v6/xgb_model_v6.json')

## Load Test Data and Supporting Data

Load other datasets as necessary, this is a placeholder from week 2 model Remove comment when done

In [3]:
app_test = pd.read_csv('/kaggle/input/home-credit-default-risk/application_test.csv')
prev_app = pd.read_csv('/kaggle/input/home-credit-default-risk/previous_application.csv')
pos_cash = pd.read_csv('/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv')
install_pmts = pd.read_csv('/kaggle/input/home-credit-default-risk/installments_payments.csv')
cc_bal = pd.read_csv('/kaggle/input/home-credit-default-risk/credit_card_balance.csv')
bureau = pd.read_csv('/kaggle/input/home-credit-default-risk/bureau.csv')

## Preprocessing

The goal in this section is to combine the test features with the relevant test features from our supporting datasets. We need the same features in the test dataset that our model was trained with.

In [4]:
%%time

# Previous Application
prev_app['CONTRACT_REFUSED_IND'] = (prev_app['NAME_CONTRACT_STATUS'] == 'Refused').astype(int)
prev_app['PROD_COMB_IND'] = prev_app['PRODUCT_COMBINATION'].isin(['Cash','POS mobile with interest','Card Street','Cash Street: high','Cash X-Sell: high']).astype(int)

prev_app.drop(columns={'NAME_CONTRACT_STATUS','PRODUCT_COMBINATION'}, inplace = True)

prev_app_agg_type = {
    'SK_ID_PREV' : 'count',
    'AMT_APPLICATION' : ['sum', 'mean', 'min', 'max', 'median'],
    'AMT_CREDIT' : ['sum', 'mean', 'min', 'max', 'median'],
    'RATE_DOWN_PAYMENT' : ['mean', 'min', 'max', 'median'],
    'RATE_INTEREST_PRIMARY' : ['mean', 'min', 'max', 'median'],
    'RATE_INTEREST_PRIVILEGED' : ['mean', 'min', 'max', 'median'],
    'CONTRACT_REFUSED_IND' : ['median', 'count'],
    'PROD_COMB_IND' : ['median', 'count']

}

prev_app_agg = prev_app.groupby('SK_ID_CURR').agg(prev_app_agg_type).reset_index()

prev_app_agg.columns = ['_'.join(col).strip() for col in prev_app_agg.columns.values]

for col in prev_app_agg.columns[1:]:
    prev_app_agg.rename(columns = {col : 'PREV_'+col}, inplace = True)
    
prev_app_agg.rename(columns = {'SK_ID_CURR_' : 'SK_ID_CURR'}, inplace = True)




# POS Cash
pos_cash.drop(columns=('NAME_CONTRACT_STATUS'), inplace = True)

pos_agg_type = {
    'SK_ID_PREV' : 'count',
    'MONTHS_BALANCE' : ['mean', 'min', 'max', 'median'],
    'CNT_INSTALMENT' : ['mean', 'min', 'max', 'median'],
    'CNT_INSTALMENT_FUTURE' : ['mean', 'min', 'max', 'median'],
    'SK_DPD' : ['mean', 'min', 'max', 'median'],
    'SK_DPD_DEF' : ['mean', 'min', 'max', 'median'],
}

pos_cash_agg = pos_cash.groupby('SK_ID_CURR').agg(pos_agg_type).reset_index()

pos_cash_agg.columns = ['_'.join(col).strip() for col in pos_cash_agg.columns.values]

pos_cash_agg['INSTAL_RATIO'] = (pos_cash_agg['CNT_INSTALMENT_mean'] / np.where(pos_cash_agg['CNT_INSTALMENT_FUTURE_mean'] != 0, pos_cash_agg['CNT_INSTALMENT_FUTURE_mean'], 1)).fillna(0)

for col in pos_cash_agg.columns[1:]:
    pos_cash_agg.rename(columns = {col : 'POS_'+col}, inplace = True)
    
pos_cash_agg.fillna(0, inplace = True)

pos_cash_agg.rename(columns = {'SK_ID_CURR_' : 'SK_ID_CURR'}, inplace = True)




# Credit Card Balance
cc_bal['USAGE_RATIO'] = (cc_bal['AMT_BALANCE'] / np.where(cc_bal['AMT_CREDIT_LIMIT_ACTUAL'] != 0 , cc_bal['AMT_CREDIT_LIMIT_ACTUAL'],1)).fillna(0)
cc_bal['INSTALL_MATURE_AMT'] = (cc_bal['AMT_BALANCE'] / np.where(cc_bal['CNT_INSTALMENT_MATURE_CUM'] != 0 , cc_bal['CNT_INSTALMENT_MATURE_CUM'],1)).fillna(0)

cc_agg_type = {
    'SK_ID_PREV' : 'count',
    'MONTHS_BALANCE' : ['sum', 'mean', 'min', 'max', 'median'],
    'AMT_BALANCE' : ['sum', 'mean', 'min', 'max', 'median'],
    'AMT_CREDIT_LIMIT_ACTUAL' : ['sum', 'mean', 'min', 'max', 'median'],
    'AMT_DRAWINGS_ATM_CURRENT' : ['sum', 'mean', 'min', 'max', 'median'],
    'AMT_DRAWINGS_CURRENT' : ['sum', 'mean', 'min', 'max', 'median'],
    'AMT_DRAWINGS_OTHER_CURRENT' : ['sum', 'mean', 'min', 'max', 'median'],
    'AMT_DRAWINGS_POS_CURRENT' : ['sum', 'mean', 'min', 'max', 'median'],
    'AMT_INST_MIN_REGULARITY' : ['sum', 'mean', 'min', 'max', 'median'],
    'AMT_PAYMENT_CURRENT' : ['sum', 'mean', 'min', 'max', 'median'],
    'AMT_PAYMENT_TOTAL_CURRENT' : ['sum', 'mean', 'min', 'max', 'median'],
    'AMT_RECEIVABLE_PRINCIPAL' : ['sum', 'mean', 'min', 'max', 'median'],
    'AMT_RECIVABLE' : ['sum', 'mean', 'min', 'max', 'median'],
    'AMT_TOTAL_RECEIVABLE' : ['sum', 'mean', 'min', 'max', 'median'],
    'CNT_DRAWINGS_ATM_CURRENT' : ['sum', 'mean', 'min', 'max', 'median'],
    'CNT_DRAWINGS_CURRENT' : ['sum', 'mean', 'min', 'max', 'median'],
    'CNT_DRAWINGS_OTHER_CURRENT' : ['sum', 'mean', 'min', 'max', 'median'],
    'CNT_DRAWINGS_POS_CURRENT' : ['sum', 'mean', 'min', 'max', 'median'],
    'CNT_INSTALMENT_MATURE_CUM' : ['sum', 'mean', 'min', 'max', 'median'],
    'SK_DPD' : ['sum', 'mean', 'min', 'max', 'median'],
    'SK_DPD_DEF' : ['sum', 'mean', 'min', 'max', 'median'],
    'USAGE_RATIO' : ['sum', 'mean', 'min', 'max', 'median'],
    'INSTALL_MATURE_AMT' : ['sum', 'mean', 'min', 'max', 'median'],
}

cc_bal_agg = cc_bal.groupby('SK_ID_CURR').agg(cc_agg_type).reset_index()

cc_bal_agg.columns = ['_'.join(col).strip() for col in cc_bal_agg.columns.values]

for col in cc_bal_agg.columns[1:]:
    cc_bal_agg.rename(columns = {col : 'CC_'+col}, inplace = True)

cc_bal_agg.fillna(0, inplace = True)

cc_bal_agg.rename(columns = {'SK_ID_CURR_' : 'SK_ID_CURR'}, inplace = True)




# Installments Payments
install_pmts['DAYS_PMT_DIFF'] = (install_pmts['DAYS_ENTRY_PAYMENT'] - install_pmts['DAYS_INSTALMENT']).fillna(0)
install_pmts['ON_TIME_COUNT'] = np.where(install_pmts['DAYS_PMT_DIFF'] <= 0,1,0)

inst_agg_type = {
    'SK_ID_PREV' : 'count',
    'NUM_INSTALMENT_VERSION' : ['mean','min','max','median'],
    'NUM_INSTALMENT_NUMBER' : ['mean','min','max','median'],
    'DAYS_INSTALMENT' : ['mean','min','max','median'],
    'DAYS_ENTRY_PAYMENT' : ['mean','min','max','median'],
    'AMT_INSTALMENT' : ['mean','min','max','median'],
    'AMT_PAYMENT' : ['mean','min','max','median'],
    'DAYS_PMT_DIFF' : ['mean','min','max','median'],
    'ON_TIME_COUNT' : ['sum','mean','min','max','median'],
}

inst_pmts_agg = install_pmts.groupby('SK_ID_CURR').agg(inst_agg_type).reset_index()

inst_pmts_agg.columns = ['_'.join(col).strip() for col in inst_pmts_agg.columns.values]

for col in inst_pmts_agg.columns[1:]:
    inst_pmts_agg.rename(columns = {col : 'INST_'+col}, inplace = True)

inst_pmts_agg['INST_ON_TIME_RATIO'] = (inst_pmts_agg['INST_ON_TIME_COUNT_sum'] / np.where(inst_pmts_agg['INST_NUM_INSTALMENT_NUMBER_max'] != 0 ,inst_pmts_agg['INST_NUM_INSTALMENT_NUMBER_max'],1)).fillna(0)
inst_pmts_agg['INST_MEAN_PAY_INSTAL_RATIO'] = (inst_pmts_agg['INST_AMT_PAYMENT_mean'] / np.where(inst_pmts_agg['INST_AMT_INSTALMENT_mean'] != 0 ,inst_pmts_agg['INST_AMT_INSTALMENT_mean'],1)).fillna(0)

inst_pmts_agg.fillna(0, inplace = True)

inst_pmts_agg.rename(columns = {'SK_ID_CURR_' : 'SK_ID_CURR'}, inplace = True)




# Bureau
bureau_agg_type = {
    'SK_ID_BUREAU': 'count',
    'CREDIT_CURRENCY': lambda x: x.mode()[0] if not x.mode().empty else 'N/A',
    'DAYS_CREDIT' : ['sum','mean','min','max','median'],
    'CREDIT_DAY_OVERDUE': ['sum','mean','min','max','median'],
    'DAYS_CREDIT_ENDDATE': ['sum','mean','min','max','median'],
    'DAYS_ENDDATE_FACT': ['sum','mean','min','max','median'],
    'AMT_CREDIT_MAX_OVERDUE': ['sum','mean','min','max','median'],
    'CNT_CREDIT_PROLONG': ['sum','mean','min','max','median'],
    'AMT_CREDIT_SUM': ['sum','mean','min','max','median'],
    'AMT_CREDIT_SUM_DEBT': ['sum','mean','min','max','median'],
    'AMT_CREDIT_SUM_LIMIT': ['sum','mean','min','max','median'],
    'AMT_CREDIT_SUM_OVERDUE': ['sum','mean','min','max','median'],
    'CREDIT_TYPE': lambda x: x.mode()[0] if not x.mode().empty else np.nan,
    'DAYS_CREDIT_UPDATE': ['sum','mean','min','max','median'],
    'AMT_ANNUITY': ['sum','mean','min','max','median']   
}

bureau_agg = bureau.groupby('SK_ID_CURR').agg(bureau_agg_type).reset_index()

bureau_agg.columns = ['_'.join(col).strip() for col in bureau_agg.columns.values]

for col in bureau_agg.columns[1:]:
    bureau_agg.rename(columns = {col : 'BUR_'+col}, inplace = True)
    
bureau_agg.rename(columns = {
    'BUR_CREDIT_CURRENCY_<lambda>' : 'BUR_CREDIT_CURRENCY_mode',
    'BUR_CREDIT_TYPE_<lambda>' : 'BUR_CREDIT_TYPE_mode'
},inplace = True)

bureau_agg['BUR_DEBT_RATIO'] = bureau_agg['BUR_AMT_CREDIT_SUM_DEBT_sum'] / np.where(bureau_agg['BUR_AMT_CREDIT_SUM_sum'] != 0, bureau_agg['BUR_AMT_CREDIT_SUM_sum'], 1)

for col in bureau_agg.columns:
    if bureau_agg[col].dtype in (int,float):
        bureau_agg[col].fillna(0, inplace = True)
        
bureau_agg.rename(columns = {'SK_ID_CURR_' : 'SK_ID_CURR'}, inplace = True)

CPU times: user 2min 53s, sys: 2.54 s, total: 2min 56s
Wall time: 2min 55s


# Reduce Memory / Create Test Dataset

Each aggregated dataset is stored in a new dataframe. Considering the size of some of these datasets, it is best to remove the source entirely

In [5]:
del install_pmts
del prev_app
del pos_cash
del cc_bal
del bureau

In [6]:
%%time

test = app_test.merge(prev_app_agg, on = 'SK_ID_CURR', how = 'left')
print('1 Merge Complete')

test = test.merge(pos_cash_agg, on = 'SK_ID_CURR', how = 'left')
print('2 Merges Complete')

test = test.merge(cc_bal_agg, on = 'SK_ID_CURR', how = 'left')
print('3 Merges Complete')

test = test.merge(inst_pmts_agg, on = 'SK_ID_CURR', how = 'left')
print('4 Merges Complete')

test = test.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')
print('All 5 Merges Completed')

1 Merge Complete
2 Merges Complete
3 Merges Complete
4 Merges Complete
All 5 Merges Completed
CPU times: user 481 ms, sys: 4.03 ms, total: 485 ms
Wall time: 483 ms


# Post-Merge Feature Engineering

In [7]:
## application test
test['LOG_INCOME_CREDIT_RATIO'] = np.log((test['AMT_INCOME_TOTAL'] / np.where(test['AMT_CREDIT'] != 0 ,test['AMT_CREDIT'],1)).fillna(0))
test['CREDIT_GOODS_RATIO'] = (test['AMT_CREDIT'] / np.where(test['AMT_GOODS_PRICE'] != 0 ,test['AMT_GOODS_PRICE'],1)).fillna(0)
test['CODE_GENDER'] = test['CODE_GENDER'].isin(['F']).astype(int)
test['FLAG_OWN_CAR'] = test['FLAG_OWN_CAR'].isin(['Y']).astype(int)
test['LOG_AMT_INCOME_TOTAL'] = np.log(test['AMT_INCOME_TOTAL'])
test['CREDIT_ANNUITY_RATIO'] = (test['AMT_CREDIT'] / np.where(test['AMT_ANNUITY'] != 0 ,test['AMT_ANNUITY'],1)).fillna(0)

## application train + previous application
test['PREV_GOODS_APPSUM'] = test['AMT_GOODS_PRICE'] / np.where(test['PREV_AMT_APPLICATION_sum'] != 0, test['PREV_AMT_APPLICATION_sum'], 1)
test['PREV_GOODS_CREDSUM'] = test['AMT_GOODS_PRICE'] / np.where(test['PREV_AMT_CREDIT_sum'] != 0, test['PREV_AMT_CREDIT_sum'], 1)
test['PREV_CREDIT_APPSUM'] = test['AMT_CREDIT'] / np.where(test['PREV_AMT_APPLICATION_sum'] != 0, test['PREV_AMT_APPLICATION_sum'], 1)
test['LOG_PREV_INC_APPSUM'] = np.log(test['AMT_INCOME_TOTAL'] / np.where(test['PREV_AMT_APPLICATION_sum'] != 0, test['PREV_AMT_APPLICATION_sum'], 1))
test['PREV_INC_CREDSUM'] = test['AMT_INCOME_TOTAL'] / np.where(test['PREV_AMT_CREDIT_sum'] != 0, test['PREV_AMT_CREDIT_sum'], 1)

## application train + POS CASH
test['POS_INST_F_GOODS'] = test['AMT_GOODS_PRICE'] / np.where(test['POS_CNT_INSTALMENT_FUTURE_mean'] != 0, test['POS_CNT_INSTALMENT_FUTURE_mean'], 1)

## application train + Installments Payments
test['INST_GOODS_PMT'] = np.log(abs(test['AMT_GOODS_PRICE'] / np.where(test['INST_AMT_PAYMENT_mean'] != 0, test['INST_AMT_PAYMENT_mean'], 1)))

## application train +  Credit Card Balance
test['CC_GOODS_INST_MAT'] = test['AMT_GOODS_PRICE'] / np.where(test['CC_INSTALL_MATURE_AMT_mean'] != 0, test['CC_INSTALL_MATURE_AMT_mean'], 1)

## application train + bureau
test['BUR_CRED_ANN_RATIO'] = test['BUR_AMT_CREDIT_SUM_sum'] / np.where(test['BUR_AMT_ANNUITY_sum'] != 0, test['BUR_AMT_ANNUITY_sum'], 1)



# Fix Indicators being converted to floats post-merge
test['PREV_CONTRACT_REFUSED_IND_median'].fillna(0, inplace = True)
test['PREV_PROD_COMB_IND_median'].fillna(0, inplace = True)

for column in test.columns:
    if test[column].dtype == float:
        unique_values = test[column].unique()
        if set(unique_values) == {0.0, 1.0}:
            test[column] = test[column].astype(int)

In [8]:
X_test = preprocessor.transform(test)

X_test.shape

(48744, 199)

## Generate Predictions

In [9]:
test_probs = model.predict_proba(X_test)

test_probs.shape

(48744, 2)

## Submission

In [10]:
submission = pd.read_csv('/kaggle/input/home-credit-default-risk/sample_submission.csv')

submission.head()

,SK_ID_CURR,TARGET
0,100001,0.5
1,100005,0.5
2,100013,0.5
3,100028,0.5
4,100038,0.5


In [11]:
submission.TARGET = test_probs[:,1]

In [12]:
submission.to_csv('submission.csv', header = True, index = False)

# Final Model Score and Closing Notes

## Private Score: **78.274%**
## Public Score: **78.425%**


# Final Notes

During this capstone project I was able to get hands on experience with large datasets, and was sucessfully able to identify loan applicants who are at a high risk of defaulting on their loans. Using the ROC AUC score, we achieved scores as high as 78.425% which is a calculated measure between sensitivity and specificity. The ROC AUC score is primarily a measure that accounts for true positives and false positives.

In addition to experience building complex models in python, I was also able to experience more complex tasks like aggregating supplementary data, testing new models, as well as various methods of hyperparameter tuning. My primary take-away from this project is the importance of developing comprehensive exploratory data analysis to understand key attributes of our dataset. Exploratory data analysis and feature engineering are responsible for the majority of our progress each week, however, on some occasions it was worthwhile exploring various transformations for our features to imporve our model's performance.

# END